In [80]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [81]:
xls = pd.ExcelFile(
    # "./data/Student and Faculty Commons Survey_February 16, 2022_09.17.xlsx"
    "./data/Student and Faculty Commons Survey_February 16, 2022_09.17.xlsx"
)

data = pd.read_excel( xls, "Student and Faculty Commons Sur" )

useable_data = data.drop( [0, 1] )
useable_data = useable_data.drop( data.columns[0:17], axis=1 )

intersections = data[ ["intersection", "columnName"] ]

useable_data.fillna('No Response', inplace=True)    
intersections.fillna("blank", inplace=True)    


In [82]:
def render_graph(question, figure):
    traces = []
    options = len(useable_data[question])

    values = np.full( options, 2 )


    buttons = []

    for i, intersection in enumerate(intersections["columnName"]):

        if not intersection == "blank":

            fig = figure(question, intersection, values, i )
            traces.append(fig)


            rows = intersections["columnName"] == intersection
            value = intersections.loc[ rows, "intersection" ].item()


            visibility = np.full( options, False )
            visibility[i] = True

            buttons.append(dict(label=str(value),
                        method="update",
                        args=[{"visible":visibility},
                              {"title":str(value)}]))

    fig = go.Figure(data=traces)

    fig.update_layout(
        updatemenus=[
            dict(
                direction="down",
                active=0,
                x=0.57,
                y=1.2,
                buttons=list(buttons)
            )
        ])    

    return fig

In [83]:
def render_sundial(question, intersection, values, i):
    return px.sunburst(useable_data, path=[question, intersection], values=values ).update_traces(visible=True if i==0 else False).data[0]

def render_bar(question, intersection, values, i):
    print(useable_data[intersection])
    return px.bar(useable_data, x=question, y=values, ).update_traces(visible=True if i==0 else False).data[0]
    


In [84]:

xls = pd.ExcelFile(
    # "./data/Student and Faculty Commons Survey_February 16, 2022_09.17.xlsx"
    "./data/temp.xlsx"
)

data = pd.read_excel( xls, "Sheet1" )

render_graph("Q22", render_bar)

# test = px.bar(useable_data, x="Q22", y="")

2         No Response
3               White
4     Native American
5               White
6         No Response
7         White,Asian
8         No Response
9               Asian
10              Asian
11              Asian
12        No Response
13        White,Asian
14        No Response
Name: Q2, dtype: object
2      No Response
3      No Response
4           Female
5       Non-Binary
6      No Response
7             Male
8      No Response
9     Gender Fluid
10          Female
11          Female
12     No Response
13            Male
14     No Response
Name: Q4, dtype: object
2      No Response
3      No Response
4      No Response
5     Heterosexual
6      No Response
7     Heterosexual
8      No Response
9         Bisexual
10        Bisexual
11        Bisexual
12     No Response
13    Heterosexual
14     No Response
Name: Q8, dtype: object
